In [1]:
import numpy as np
import pandas as pd

import os

In [2]:
monthly_data = pd.read_csv(os.path.join(os.getcwd(),
                                        'data',
                                        'input',
                                        'OR_Databank_Q1_2007-Q4_2024.csv'),
                            header=0)

monthly_data.head(1)

C:\Users\ellin\AppData\Local\Temp\ipykernel_25668\1384618058.py:1: DtypeWarning: Columns (10,15,28,33,34,39,40,41,42,43,44,45,58,59,62,63,64,65,66,67,68,70,72,73,75,77,94,98,99,100,101,103,110,111,112,113,114,115,130,132,147,158,160,161,162,164,166,167,168) have mixed types. Specify dtype option on import or set low_memory=False.
  monthly_data = pd.read_csv(os.path.join(os.getcwd(),


,AHA ID,Hospital Name,Hospital Short Name,Type,Critical Access,Month,Quarter,Year,Available Beds,Licensed Beds,...,Gross Patient Accounts Receivable (Medicare),Gross Patient Accounts Receivable (Medicaid),Gross Patient Accounts Receivable (Self Pay),Gross Patient Accounts Receivable (Commercial),Gross Patient Accounts Receivable (Others),Gross Patient Accounts Receivable,Uncompensated Care,Inpatient Discharges,Total Operating Revenue,Total Revenue
0,6920770,Adventist Health Columbia Gorge Medical Center,Adventist Columbia Gorge Med Ctr,B,False,1,1,2024,43,49,...,"6,312,933","3,151,689","4,301,845","18,645,968","1,363,000","33,775,435",406239,136,9234294,9198710


In [3]:
categorical_cols = [
    'Hospital Name',
    'Hospital Short Name',
    'Type',
    'Critical Access',
] # excluding AHA Id because its a grouping column
proportion_cols = [
    r'Operating Margin %',
    r'total margin %',
    r'CCR',
    r'charity care %',
    r'bad debt %',
    r'Medicaid %',
    r'Medicare %',
    r'Commercial %',
    r'Self Pay %',
]

sum_agg_cols = list(set(monthly_data.columns).difference(set(categorical_cols+proportion_cols+['Month', 'Year', 'Quarter', 'AHA ID'])))

In [4]:
for col in sum_agg_cols:
    print(f'{col}: {monthly_data[col].dtype}')

Contractuals of Other: object
Total Outpatient Visits: object
Physician Payroll Amount: object
Total Margin : object
Charges of DPU (Medicaid): object
Operating Margin: object
Total Discharges (Self Pay): int64
Total Contractual Allowances (Medicare): object
Facility Payroll Hours: object
Patient Days of DPU (Others): object
Discharges of DPU (Others): int64
Patient Days of Swing Bed (Others): int64
Total Patient Days (Medicaid): object
Capitated Payments (Medicare): object
Total Charges (Medicaid): object
Facility Payroll Amount: object
Total Contractual Allowances (Self Pay): object
Capitated Payments (Others): object
Total Contractual Allowances (Others): object
Total Contractual Allowances (Medicaid): object
Net Patient Revenue (Commercial): object
Patient Days of Subacute & LTC (Others): int64
Charges of Acute Inpatient (Others): object
Patient Days of Swing Bed (Medicare): int64
Discharges of Swing Bed (Medicaid): int64
Charges of DPU (Others): object
Charges of Acute Outpatient 

In [5]:
print(monthly_data['Charges of Swing Bed (Medicaid)'].unique())

['0' '36,892' '4,133' ... '2703' '1485' '9827']


In [6]:
monthly_data[sum_agg_cols] = monthly_data[sum_agg_cols].apply(pd.to_numeric, errors='coerce')
print((monthly_data.isna().sum() / monthly_data.shape[0])[(monthly_data.isna().sum() / monthly_data.shape[0]) > 0.0])

Discharges of Acute Inpatient (Medicare)          0.002817
Discharges of Acute Inpatient (Commercial)        0.775569
Discharges of Acute Inpatient                     0.029658
Discharges of Swing Bed (Commercial)              0.775569
Discharges of Subacute & LTC (Commercial)         0.775569
                                                    ...   
Gross Patient Accounts Receivable (Medicaid)      0.196494
Gross Patient Accounts Receivable (Self Pay)      0.196338
Gross Patient Accounts Receivable (Commercial)    0.966195
Gross Patient Accounts Receivable (Others)        0.179905
Gross Patient Accounts Receivable                 0.196494
Length: 153, dtype: float64


In [ ]:
agg_dict = {key:'sum' for key in sum_agg_cols}
agg_dict.update({key:(lambda s: s.mode().iloc[0]) for key in categorical_cols})
# agg_dict.update({key:'mean' for key in proportion_cols})

quarterly_sums = monthly_data\
    .drop(columns=['Month'])\
    .groupby(['AHA ID', 'Year', 'Quarter']).agg(agg_dict).reset_index()

quarterly_sums.head()

,AHA ID,Year,Quarter,Contractuals of Other,Total Outpatient Visits,Physician Payroll Amount,Total Margin,Charges of DPU (Medicaid),Operating Margin,Total Discharges (Self Pay),...,Charges of Home Health (Self Pay),Charges of Home Health,Patient Days of Subacute & LTC (Self Pay),Patient Days of Acute Inpatient (Commercial),Discharges of Swing Bed (Self Pay),Total Discharges (Others),Hospital Name,Hospital Short Name,Type,Critical Access
0,6920003,2007,1,0.0,46742.0,2368587.0,706200.0,1919086.0,-997758.0,445,...,0.0,0.0,0,0.0,0,2086,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False
1,6920003,2007,2,0.0,48120.0,1949079.0,9848484.0,1889494.0,7620054.0,488,...,0.0,0.0,0,0.0,0,2209,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False
2,6920003,2007,3,0.0,45013.0,2368363.0,9418875.0,2698637.0,9598107.0,523,...,0.0,0.0,0,0.0,0,2271,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False
3,6920003,2007,4,0.0,43286.0,2101771.0,3486429.0,1858391.0,3599673.0,415,...,0.0,0.0,0,0.0,0,2137,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False
4,6920003,2008,1,0.0,45035.0,2539164.0,-1080819.0,1686042.0,3713899.0,400,...,0.0,0.0,0,0.0,0,2210,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False


In [8]:
print(len(list(quarterly_sums.columns)) - len(list(quarterly_sums.columns.unique())))
print(quarterly_sums.columns[quarterly_sums.columns.duplicated()])

0
Index([], dtype='object')


In [9]:
print((quarterly_sums.isna().sum() / quarterly_sums.shape[0])[(quarterly_sums.isna().sum() / quarterly_sums.shape[0]) > 0.0])

Series([], dtype: float64)


In [10]:
print(list(set(monthly_data.columns).difference(set(quarterly_sums.columns))))

['Month']


In [11]:
quarterly_sums = quarterly_sums[list(monthly_data.columns)[:5] + list(monthly_data.columns)[6:]]
quarterly_sums.head(1)

,AHA ID,Hospital Name,Hospital Short Name,Type,Critical Access,Quarter,Year,Available Beds,Licensed Beds,Discharges of Acute Inpatient (Medicare),...,Gross Patient Accounts Receivable (Medicare),Gross Patient Accounts Receivable (Medicaid),Gross Patient Accounts Receivable (Self Pay),Gross Patient Accounts Receivable (Commercial),Gross Patient Accounts Receivable (Others),Gross Patient Accounts Receivable,Uncompensated Care,Inpatient Discharges,Total Operating Revenue,Total Revenue
0,6920003,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False,1,2007,1203,1662,771.0,...,50749312.0,75957402.0,54907221.0,0.0,281437663.0,463051598.0,20221600,4663,98490357,100194315


In [12]:
quarterly_sums.to_csv(os.path.join(os.getcwd(), 'data', 'input', 'OR_quarterly_aggregation.csv'), index=False)